[Group 62] Machines of ML

Frimann Bergvik Garmann         - 527245
Brage Bergsmyr                  - 514881
Magnus Christian Kvist Jacobsen - 506626

In [ ]:
!pip install catboost
!pip install geopandas
!pip install folium
!pip install matplotlib
!pip install mapclassify
!pip install shapely
!pip install numpy
!pip install sklearn
!pip install xgboost
!pip install catboost
!pip install scipy
!pip install lightgbm
!pip install h2o
!pip install pandas
!pip install dataprep
!pip install geopy
!pip install fiona
!pip install pyproj
!pip install packaging
!pip install seaborn

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import math
import geopy.distance
import matplotlib.pyplot as plt
import seaborn as sns
from dataprep import eda
import xgboost as xgb
from shapely.geometry import Point
from sklearn import datasets
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, mean_squared_log_error
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

plt.style.use("ggplot")

pd.set_option('display.max_columns', None)

In [2]:
busstops_norway                     = pd.read_csv('../data/busstops_norway.csv')
grunnkrets_age_distribution         = pd.read_csv('../data/grunnkrets_age_distribution.csv')
grunnkrets_households_num_persons   = pd.read_csv('../data/grunnkrets_households_num_persons.csv')
grunnkrets_income_households        = pd.read_csv('../data/grunnkrets_income_households.csv')
grunnkrets_norway_stripped          = pd.read_csv('../data/grunnkrets_norway_stripped.csv')
plaace_hierarchy                    = pd.read_csv('../data/plaace_hierarchy.csv')
sample_submission                   = pd.read_csv('../data/sample_submission.csv')
stores_extra                        = pd.read_csv('../data/stores_extra.csv')
stores_test                         = pd.read_csv('../data/stores_test.csv')
stores_train                        = pd.read_csv('../data/stores_train.csv')

#eda.create_report(stores_train)

In [3]:
# helper functions
def left_merge(X, Z, on):
    return pd.merge(X, Z, how='left', on=on)

def replace_missing(X, column, replacement):
    _X = X.copy()
    _X[column] = _X[column].replace(np.nan, replacement)
    return _X

def remove_nan_rows(X, column):
    _X = X.copy()
    return _X[_X[column].notna()]

def remove_column(X, column):
    _X = X.copy()
    return _X.drop(column, axis=1)

def remove_year(X):
    X_copy = X.copy()
    columns = X.columns
    X_copy = X_copy.loc[X_copy.groupby('grunnkrets_id')['year'].idxmax()]
    X_copy = X_copy.drop(columns=['year'])
    #X_copy = X_copy.groupby(['grunnkrets_id'], as_index=False).agg(dict)
    return X_copy

In [4]:
# Feature functions
def sum_people(X):
    y_grunnkrets = X.grunnkrets_id
    X_copy = X.drop(columns=['grunnkrets_id'])
    X_sum = X_copy.sum(axis=1)
    return pd.merge(y_grunnkrets.rename('grunnkrets_id'), X_sum.rename('sum_people'), left_index=True, right_index=True)

def sum_stores(X):
    X_sum = X['grunnkrets_id'].groupby('grunnkrets_id').sum()
    return X_sum

def jeg_er_lei_meg(X):
    # Adds a num_of_stores to the district

    districts=X.loc[:,"district_name"].unique()
    dict={"district_name":[],"num_of_stores":[]}
    for district in districts:
        num_of_stores=(X.loc[X["district_name"] == district])["store_id"].count()
        dict["district_name"]+=[district]
        dict["num_of_stores"]+=[num_of_stores]
    return pd.DataFrame.from_dict(dict)


def clostest_rival(df, shop_id):
    #hente ut samme hierarky
    #sammenligne distansen til nærmeste

    hierarchy = df.loc[df["store_id"] == shop_id]["plaace_hierarchy_id"].values
    coords_1 = df.loc[df["store_id"] == shop_id][["lon","lat"]].values
    entry_index= df.loc[df["store_id"] == shop_id].index
    
    check = df.loc[df["plaace_hierarchy_id"] == hierarchy[0]]
    #print(check)
    distance=math.inf
    best_row=None
    #finds=""
    for index, row in check.iterrows():
        coords_2 = row[["lon","lat"]].values
        current_distance= geopy.distance.geodesic(coords_1, coords_2).km

        if (current_distance < distance and index != entry_index):
            distance=current_distance
            best_row=row
     #       finds+=str(row["store_id"])
    #print(distance, "finds",finds)
    return distance, best_row


def add_avg_revenue_municipality(X): #ALL MY HOMIES HATE THIS FUNCTION
    districts=X.loc[:,"municipality_name"].unique()
    dict={"municipality_name":[],"mean":[]}
    for district in districts:
        mean = (X.loc[X["municipality_name"] == district])["revenue"].mean()
        dict["district_name"]+=[district]
        dict["mean"]+=[mean]
    df= pd.DataFrame.from_dict(dict)
    print(df)
    bass = pd.merge(X, df, how="left", on="municipality_name")
    X["avg_revenue_municipality"] = bass["mean"]

In [5]:
# remove rows with revenue outliers
print(stores_train.shape)
stores_train = stores_train[stores_train['revenue'] > 0.01]
print(stores_train.shape)
big_chungus = stores_train[stores_train['revenue'] > 135.0] 
stores_train = stores_train[stores_train['revenue'] < 150.0] # 180 for å fjerne øverste 5
print(stores_train.shape)

stores_train['revenue'] = np.log1p(stores_train['revenue'])

MEAN_REVENUE = stores_train["revenue"].mean()
print(MEAN_REVENUE)
#eda.create_report(big_chungus)

(12859, 12)
(12610, 12)
(12598, 12)
1.6160189057767156


In [6]:
plaace_hierarchy=plaace_hierarchy.drop(columns=["sales_channel_name"])

stores_train=left_merge(stores_train, plaace_hierarchy, on="plaace_hierarchy_id")
stores_test=left_merge(stores_test, plaace_hierarchy, on="plaace_hierarchy_id")

# removing year
grunnkrets_age_distribution = remove_year(grunnkrets_age_distribution)
grunnkrets_households_num_persons = remove_year(grunnkrets_households_num_persons)
grunnkrets_income_households = remove_year(grunnkrets_income_households)
grunnkrets_norway_stripped = remove_year(grunnkrets_norway_stripped)

stores_train = left_merge(stores_train, grunnkrets_norway_stripped[['grunnkrets_id', 'district_name', "municipality_name", "area_km2"]], on='grunnkrets_id')
stores_test = left_merge(stores_test, grunnkrets_norway_stripped[['grunnkrets_id', 'district_name',"municipality_name", "area_km2"]], on='grunnkrets_id')

stores_test.head(3)


,store_id,year,store_name,plaace_hierarchy_id,sales_channel_name,grunnkrets_id,address,lat,lon,chain_name,mall_name,lv1,lv1_desc,lv2,lv2_desc,lv3,lv3_desc,lv4,lv4_desc,district_name,municipality_name,area_km2
0,914206820-914239427-717245,2016,VÅLERENGA HALAL BURGER AS,1.1.1.0,Hamburger restaurants,3012704,STRØMSVEIEN 25 A,59.908672,10.787031,NaN,NaN,1,Dining and Experiences,1.1,Restaurant,1.1.1,Hamburger restaurants,1.1.1.0,Hamburger restaurants,Vålerenga,Oslo,0.057027
1,916789157-916823770-824309,2016,BURGER KING MYREN,1.1.1.0,Hamburger restaurants,8061401,MYREN 1,59.201467,9.588243,BURGER KING,NaN,1,Dining and Experiences,1.1,Restaurant,1.1.1,Hamburger restaurants,1.1.1.0,Hamburger restaurants,Gulset,Skien,0.165993
2,913341082-977479363-2948,2016,BURGER KING STOVNER,1.1.1.0,Hamburger restaurants,3013917,STOVNER SENTER 3,59.962146,10.924524,BURGER KING,Stovner Senter,1,Dining and Experiences,1.1,Restaurant,1.1.1,Hamburger restaurants,1.1.1.0,Hamburger restaurants,Fossum,Oslo,0.236628


In [7]:
stores_train = replace_missing(stores_train, "chain_name", "no chain")
stores_train = replace_missing(stores_train, "mall_name", "no mall")
stores_train = replace_missing(stores_train, 'district_name', 'No district')
stores_train = replace_missing(stores_train, 'municipality_name' , "No municipality_name")
stores_train = replace_missing(stores_train, 'area_km2' , 0.0)



stores_test = replace_missing(stores_test, "chain_name", "no chain")
stores_test = replace_missing(stores_test, "mall_name", "no mall")
stores_test = replace_missing(stores_test, 'district_name', 'No district')
stores_test = replace_missing(stores_test, 'municipality_name' , "No municipality_name")
stores_test = replace_missing(stores_test, 'area_km2' , 0.0)

In [8]:
def find_num_stores_district(X): # Jeg er lei meg v2.0
    districts=X.loc[:,"district_name"].unique()
    dict={"district_name":[],"num_of_stores_district":[]}
    for district in districts:
        num_of_stores=(X.loc[X["district_name"] == district])["store_id"].count()
        dict["district_name"]+=[district]
        dict["num_of_stores_district"]+=[num_of_stores]
    return pd.DataFrame.from_dict(dict)

def find_num_stores_municipality(X):
    municipalities=X.loc[:,"municipality_name"].unique()
    dict={"municipality_name":[],"num_of_stores_municipality":[]}
    for muni in municipalities:
        num_of_stores = (X.loc[X["municipality_name"] == muni])["store_id"].count()
        dict["municipality_name"]+=[muni]
        dict["num_of_stores_municipality"]+=[num_of_stores]
    return pd.DataFrame.from_dict(dict)

def find_num_stores_grunnkrets(X):
    grunnkretser=X.loc[:,"grunnkrets_id"].unique()
    dict={"grunnkrets_id":[],"num_of_stores_grunnkrets":[]}
    for grunnkrets in grunnkretser:
        num_of_stores = (X.loc[X["grunnkrets_id"] == grunnkrets])["store_id"].count()
        dict["grunnkrets_id"]+=[grunnkrets]
        dict["num_of_stores_grunnkrets"]+=[num_of_stores]
    return pd.DataFrame.from_dict(dict)

In [9]:
all_stores = stores_train.drop(columns=['revenue']).append(stores_test, ignore_index=True).append(stores_extra, ignore_index=True)
all_stores=pd.merge(all_stores,grunnkrets_norway_stripped, how="left", on="grunnkrets_id")

#distr=find_num_stores_district(all_stores)
#muni=find_num_stores_municipality(all_stores)
grunkr=find_num_stores_grunnkrets(all_stores)


#stores_train= left_merge(stores_train, distr, on="district_name")
#stores_train= left_merge(stores_train, muni, on="municipality_name")
stores_train= left_merge(stores_train, grunkr, on="grunnkrets_id")

#stores_test= left_merge(stores_test, distr, on="district_name")
#stores_test= left_merge(stores_test, muni, on="municipality_name")
stores_test= left_merge(stores_test, grunkr, on="grunnkrets_id")


In [10]:
from scipy import spatial

def mall_df(X):
    #alls=X.loc[:,"mall_name"].unique()
    #mallss = malls[np.logical_not(pd.isnull(malls))]
    all_stores = np.array(X.values)
    df69 = X.groupby('mall_name', as_index=False)["lat","lon"].mean()
    df69_nump = np.array(X.groupby('mall_name')["lat","lon"].mean())


    lat_ind = X.columns.get_loc("lat")
    lon_ind = X.columns.get_loc("lon")
    store_id_ind = X.columns.get_loc("store_id")

    dict={"store_id":[], "mall_distance":[], "closest_mall_name":[]}

    i=0
    maxxx=len(all_stores)
    for store in all_stores:
        if i%5000==0:
            print(100*(i/maxxx),"%")
        i+=1

        #distance=math.inf
        #closest_mall=None
        lat = store[lat_ind]
        lon = store[lon_ind]
        store_id=store[store_id_ind]
        coords_1 = [lat,lon]

        #A = df69_nump[spatial.KDTree(df69_nump).query(coords_1)[1]] # <-- the nearest point 
        B = spatial.KDTree(df69_nump).query(coords_1)
        distance=B[0]
        mall =df69.iloc[[B[1]]]
        dict["store_id"]+=[store_id]
        dict["mall_distance"]+=[distance]
        dict["closest_mall_name"]+=[mall.values[0][0]]
        #print(distance)
        #print(mall)
        #print("A", A)


    return pd.DataFrame.from_dict(dict)


#df69 = all_stores.groupby('mall_name')["lat","lon"].mean()

In [11]:
mall_df = mall_df(all_stores)

stores_train = pd.merge(stores_train, mall_df, how="left", on="store_id")
stores_test = pd.merge(stores_test, mall_df, how="left", on="store_id")

/tmp/ipykernel_1434626/1184942796.py:7: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df69 = X.groupby('mall_name', as_index=False)["lat","lon"].mean()
/tmp/ipykernel_1434626/1184942796.py:8: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df69_nump = np.array(X.groupby('mall_name')["lat","lon"].mean())


0.0 %
10.028279748891874 %
20.05655949778375 %
30.084839246675628 %
40.1131189955675 %
50.14139874445937 %
60.169678493351256 %
70.19795824224313 %
80.226237991135 %
90.25451774002687 %


In [12]:
def chain_info(stores_train):
    df69 = stores_train.groupby('chain_name', as_index=True)["store_id"].count()
    big_chain = df69.loc[df69 >= 1] # Frimann
    big_chain = big_chain.dropna()

    dict = {"chain_name":[], "avg_rev_big_chain":[]}

    for i in range(len(big_chain)):
        avg_rev = stores_train.loc[stores_train["chain_name"] == big_chain.index[i]]["revenue"].mean()
        dict["chain_name"]+=[big_chain.index[i]]
        dict["avg_rev_big_chain"]+=[avg_rev]
        #print(big_chain.index[i], avg_rev)
    big_chain_df = pd.DataFrame.from_dict(dict)
    return big_chain_df

In [13]:
avg_rev_big_chain= chain_info(stores_train)

stores_train = left_merge(stores_train, avg_rev_big_chain, on="chain_name")
stores_test = left_merge(stores_test, avg_rev_big_chain, on="chain_name")

stores_train = replace_missing(stores_train, 'avg_rev_big_chain' , MEAN_REVENUE)
stores_test = replace_missing(stores_test, 'avg_rev_big_chain' , MEAN_REVENUE)


In [14]:
# adds avg_rev for different hierarchy levels
def avg_rev_hierarchy_levels(X):
    lv_1_hierarkies = (X.loc[:,"lv1_desc"]).unique()
    lv_2_hierarkies = (X.loc[:,"lv2_desc"]).unique()
    lv_3_hierarkies = (X.loc[:,"lv3_desc"]).unique()
    lv_4_hierarkies = (X.loc[:,"lv4_desc"]).unique()

    dict = {"lv1_desc":[],"lv1_avg_rev":[]}
    for lv1 in lv_1_hierarkies:
        avg_rev = (X.loc[X["lv1_desc"] == lv1])["revenue"].mean()
        dict["lv1_desc"]+=[lv1]
        dict["lv1_avg_rev"]+=[avg_rev]
    lv1_df = pd.DataFrame.from_dict(dict)

    dict = {"lv2_desc":[],"lv2_avg_rev":[]}
    for lv2 in lv_2_hierarkies:
        avg_rev = (X.loc[X["lv2_desc"] == lv2])["revenue"].mean()
        dict["lv2_desc"]+=[lv2]
        dict["lv2_avg_rev"]+=[avg_rev]
    lv2_df = pd.DataFrame.from_dict(dict)

    dict = {"lv3_desc":[],"lv3_avg_rev":[]}
    for lv3 in lv_3_hierarkies:
        avg_rev = (X.loc[X["lv3_desc"] == lv3])["revenue"].mean()
        dict["lv3_desc"]+=[lv3]
        dict["lv3_avg_rev"]+=[avg_rev]
    lv3_df = pd.DataFrame.from_dict(dict)

    dict = {"lv4_desc":[],"lv4_avg_rev":[]}
    for lv4 in lv_4_hierarkies:
        avg_rev = (X.loc[X["lv4_desc"] == lv4])["revenue"].mean()
        dict["lv4_desc"]+=[lv4]
        dict["lv4_avg_rev"]+=[avg_rev]
    lv4_df = pd.DataFrame.from_dict(dict)

    return lv1_df, lv2_df, lv3_df, lv4_df


In [15]:
_,_,lv3,lv4 = avg_rev_hierarchy_levels(stores_train)

stores_train = pd.merge(stores_train, lv3, how="left", on="lv3_desc")
stores_train = pd.merge(stores_train, lv4, how="left", on="lv4_desc")

stores_test = pd.merge(stores_test, lv3, how="left", on="lv3_desc")
stores_test = pd.merge(stores_test, lv4, how="left", on="lv4_desc")

lv4.describe()

,lv4_avg_rev
count,90.000000
mean,1.639258
std,0.624691
min,0.453620
25%,1.255351
50%,1.509913
75%,1.894660
max,4.102165


In [16]:
# magnus cluster features


from scipy.spatial.distance import pdist, squareform
import sklearn
from sklearn.cluster import DBSCAN
from math import radians, sin, cos, asin, sqrt

revenues = stores_train[['store_id', 'revenue']]
all_stores = stores_train.drop(columns=['revenue']).append(stores_test, ignore_index=True).append(stores_extra, ignore_index=True)

df=all_stores[['store_id','lat','lon']]
geometry = [Point(xy) for xy in zip(df.lat, df.lon)]
gdf = gpd.GeoDataFrame(df, geometry=geometry)
latlon = all_stores[['lat','lon']]
X = np.radians(np.array(latlon, dtype='float64'))

clustering = DBSCAN(eps=2.0/6371.0, min_samples=250, metric='haversine', algorithm='ball_tree').fit(X)

y_clusters = clustering.labels_

gdf['cluster'] = y_clusters

all_stores['cluster'] = y_clusters
gdf=gdf[gdf['cluster']!=-1]
gdf.crs = "EPSG:9672"
print(gdf.loc[gdf['cluster'].idxmax()])

cluster_center = all_stores.groupby('cluster')['lat','lon'].mean()

cluster_center.rename(columns = {'lat':'center_lat', 'lon':'center_lon'}, inplace = True)

all_stores = left_merge(all_stores, cluster_center, on='cluster')

all_stores
def distance(lat1, lon1, lat2, lon2, cluster):
    if cluster == -1:
        return 15
    else:
        return np.linalg.norm(np.array([lat1,lon1])-np.array([lat2, lon2]))

from math import radians, cos, sin, asin, sqrt
def haversine(lon1, lat1, lon2, lat2, cluster):
    max_distance = 20
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    if cluster == -1:
        return max_distance
    else:
        # convert decimal degrees to radians 
        lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
        # haversine formula 
        dlon = lon2 - lon1 
        dlat = lat2 - lat1 
        a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
        c = 2 * asin(sqrt(a)) 
        km = 6371 * c
        if km >= max_distance:
            return max_distance
        else:
            return km
    
all_stores['center_distance'] = np.exp(all_stores[['lon', 'lat','center_lon','center_lat', 'cluster']].apply(lambda row: haversine(row['lon'], row['lat'], row['center_lon'], row['center_lat'], row['cluster']), axis=1))

stores_train = all_stores[all_stores['store_id'].isin(stores_train['store_id'])]
stores_train = left_merge(stores_train, revenues, on='store_id')
stores_test = all_stores[all_stores['store_id'].isin(stores_test['store_id'])]
stores_extra = all_stores[all_stores['store_id'].isin(stores_extra['store_id'])]
df=all_stores[['store_id','lat','lon', 'cluster', 'center_distance', 'store_name']]
geometry = [Point(xy) for xy in zip(df.lat, df.lon)]
gdf = gpd.GeoDataFrame(df, geometry=geometry)
gdf=gdf[gdf['cluster']!=-1]
sample=gdf.sample(frac=0.25)
sample.crs = "EPSG:9672"
#sample.explore("cluster", marker_type="circle_marker",marker_kwds={"radius":8}, legend=False)

stores_train.head(5)


# TODO Add revenue for rival
print("nei")


store_id                   918023925-918087567-872230
lat                                         68.779439
lon                                         16.566267
geometry    POINT (68.77943861059249 16.566267033764)
cluster                                            33
Name: 15, dtype: object


/tmp/ipykernel_1434626/1164115309.py:29: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  cluster_center = all_stores.groupby('cluster')['lat','lon'].mean()


nei


In [17]:
# magnus rival features

# find closest rival in stores_train (those with revenue)
from scipy import spatial

from math import radians, cos, sin, asin, sqrt
def haversine(lon1, lat1, lon2, lat2):
    max_distance = 20
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    return 6371 * c

def haversine_vec(coord1, coord2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    lat1, lon1 = coord1[0], coord1[1]
    lat2, lon2 = coord2[0], coord2[1]
    # convert decimal degrees to radians 
    return haversine(lon1, lat1, lon2, lat2)
'''

def calc_rivals_fast(X, df_with_revenue):
    
    X_copy = X.copy()
    X_copy['closest_rival'] = np.nan
    plaace_hierarchies = X['plaace_hierarchy_id'].unique()

    plaace_hierarchies = plaace_hierarchies[~pd.isnull(plaace_hierarchies)]
    for hierarchy in plaace_hierarchies:
        X_sub = X_copy[X_copy['plaace_hierarchy_id'] == hierarchy]
        df_sub = df_with_revenue[df_with_revenue['plaace_hierarchy_id'] == hierarchy]
        X_sub.apply(lambda col: col.drop_duplicates().reset_index(drop=True))
        df_sub.apply(lambda col: col.drop_duplicates().reset_index(drop=True))

        mat = spatial.distance.cdist(X_sub[['lat','lon']], 
                              df_sub[['lat','lon']], metric=haversine_vec)
        
        new_df = pd.DataFrame(mat, index=X_sub['store_id'], columns=df_sub['store_id'])
        stores = new_df.columns
        for store_id in stores:
            new_df.loc[store_id, store_id] = math.inf

        
        arr = new_df.values
        try:
            closest = np.where(arr == np.nanmin(arr, axis=1)[:,None],new_df.columns,False)

            X_sub['closest'] = [i[i.astype(bool)].tolist() for i in closest]

            X_sub['closest_rival'] = [i[i.astype(bool)] for i in closest]

            c = X_copy.closest_rival
            c.update(X_sub.closest_rival)
            X_copy['closest_rival'] = c
            X_copy.reset_index(drop=True)
            X_copy.loc[X_copy['plaace_hierarchy_id'] == hierarchy, 'closest']= X_sub['closest']
        except ValueError:
            continue

    return X_copy[['store_id', 'closest_rival']]

rival_col = 'rival_id'
all_stores = stores_train.drop(columns=['revenue']).append(stores_test, ignore_index=True).append(stores_extra, ignore_index=True)

closest_rival = calc_rivals_fast(all_stores, stores_train)

# clean id-s, get single closest store
closest_rival[rival_col] = closest_rival['closest_rival'].astype(str)

closest_rival[rival_col] = closest_rival[rival_col].apply(lambda x: x.split(' ')[0])
closest_rival[rival_col] = closest_rival[rival_col].apply(lambda x: x.replace('[', '')).apply(lambda x: x.replace(']', ''))

closest_rival[rival_col] = closest_rival[rival_col].apply(lambda x: x.replace("'", ''))

all_stores[rival_col] = closest_rival[rival_col]

rival_stores = stores_train.copy()

rival_stores['rival_lat'] = rival_stores['lat']
rival_stores['rival_lon'] = rival_stores['lon']
rival_stores['rival_revenue'] = rival_stores['revenue']
rival_stores['rival_chain'] = rival_stores['chain_name']
rival_stores[rival_col] = rival_stores['store_id']

revenue = stores_train[['store_id', 'revenue']]

all_stores = pd.merge(all_stores, rival_stores[[rival_col, 'rival_lat', 'rival_lon', 'rival_revenue', 'rival_chain']], left_on=[rival_col], right_on= [rival_col], how = 'left')
print("")
all_stores['rival_dist'] = all_stores[['lat', 'lon','rival_lat','rival_lon']].apply(lambda row: haversine(row['lon'], row['lat'], row['rival_lon'], row['rival_lat']), axis=1)

all_stores
'''
print("nei")

#stores_train = pd.merge(stores_train, all_stores[["store_id", "rival_dist"]], how="left", on="store_id")
#stores_test = pd.merge(stores_test, all_stores[["store_id", "rival_dist"]], how="left", on="store_id")

nei


In [18]:
# magnus bus features

''' 
from shapely import wkt

bsn = busstops_norway
bsn = bsn[(bsn['importance_level'] == 'Lokalt knutepunkt') | (bsn['importance_level'] == 'Annen viktig holdeplass') | (bsn['importance_level'] == 'Nasjonalt knutepunkt') | (bsn['importance_level'] == 'Regionalt knutepunkt')]
bsn['geometry'] = bsn.geometry.apply(wkt.loads)
bsn = gpd.GeoDataFrame(bsn, geometry='geometry')
bsn['lat'] = bsn.geometry.apply(lambda p: p.y)
bsn['lon'] = bsn.geometry.apply(lambda p: p.x)


# Find number of busstops within busstops_radius

# SET THIS PARAMETER IN UNIT KM!
busstops_radius = 0.5

all_stores = stores_train.drop(columns=['revenue']).append(stores_test, ignore_index=True) #.append(stores_extra, ignore_index=True)

revenue = stores_train[['store_id', 'revenue']]

def busstops_within_radius(X, busstops, radius):
    
    X_copy = X.copy()
    
    col_name = ''.join(['n_busstops_within_', str(radius), 'km'])
    
    X_copy[col_name] = np.nan

    mat = spatial.distance.cdist(X_copy[['lat','lon']], 
                          busstops[['lat','lon']], metric=haversine_vec)

    new_df = pd.DataFrame(mat, index=X_copy['store_id'], columns=busstops['busstop_id'])

    arr = new_df.values
    
    closest = np.where(arr <= (radius),new_df.columns,False)
    # Get column names ignoring false.
    X_copy[col_name] = [i[i.astype(bool)] for i in closest]
    update_stores = X_copy['store_id'].unique()
    print('hei')
    for store in update_stores:
        arry = X_copy.loc[X_copy['store_id'] == store, col_name]
        #print(arry)
        j = 0
        for i in arry:
            j = i
            continue
        #print(len(arry))
        X_copy.loc[X_copy['store_id'] == store, col_name] = len(j)
    return X_copy[['store_id', col_name]]

busstops_within_radius = busstops_within_radius(all_stores, bsn, busstops_radius)

all_stores = pd.merge(all_stores, busstops_within_radius, how='left', on='store_id')

stores_train = all_stores[all_stores['store_id'].isin(stores_train['store_id'])]
stores_test = all_stores[all_stores['store_id'].isin(stores_test['store_id'])]
#stores_extra = all_stores[all_stores['store_id'].isin(stores_extra['store_id'])]
stores_train = pd.merge(stores_train, revenue, how='left', on='store_id')
busstops_within_radius

'''

" \nfrom shapely import wkt\n\nbsn = busstops_norway\nbsn = bsn[(bsn['importance_level'] == 'Lokalt knutepunkt') | (bsn['importance_level'] == 'Annen viktig holdeplass') | (bsn['importance_level'] == 'Nasjonalt knutepunkt') | (bsn['importance_level'] == 'Regionalt knutepunkt')]\nbsn['geometry'] = bsn.geometry.apply(wkt.loads)\nbsn = gpd.GeoDataFrame(bsn, geometry='geometry')\nbsn['lat'] = bsn.geometry.apply(lambda p: p.y)\nbsn['lon'] = bsn.geometry.apply(lambda p: p.x)\n\n\n# Find number of busstops within busstops_radius\n\n# SET THIS PARAMETER IN UNIT KM!\nbusstops_radius = 0.5\n\nall_stores = stores_train.drop(columns=['revenue']).append(stores_test, ignore_index=True) #.append(stores_extra, ignore_index=True)\n\nrevenue = stores_train[['store_id', 'revenue']]\n\ndef busstops_within_radius(X, busstops, radius):\n    \n    X_copy = X.copy()\n    \n    col_name = ''.join(['n_busstops_within_', str(radius), 'km'])\n    \n    X_copy[col_name] = np.nan\n\n    mat = spatial.distance.cdist

In [19]:
#stores_train = pd.merge(stores_train, all_stores[["store_id", "rival_revenue"]], how="left", on="store_id")
#stores_test = pd.merge(stores_test, all_stores[["store_id", "rival_revenue"]], how="left", on="store_id")

In [20]:
#stores_train.loc[stores_train["cluster"] == -1].head(10)

#save id for l8r

stores_test_id = np.asarray(stores_test.store_id)

#stores_train = stores_train.replace([np.inf, -np.inf], 0, inplace=True)

remove_columns = [
    'address',
    'store_name',
    'sales_channel_name',
    'lat',
    'lon',
    #'Unnamed',
    #'distance_rival',
    'center_lat',
    'center_lon',
    'lv1',
    'lv2',
    'lv3',
    'lv4',
    'year',
    'store_id',
    'plaace_hierarchy_id',

    #'stores_in_mall',
    #'district_name',
    #'people_per_store'
]
#print(stores_train.head(10))

# data cleaning for train
for column in remove_columns:
    #print(column)
    stores_train = remove_column(stores_train, column)
    stores_test = remove_column(stores_test, column)
#stores_train = remove_column(stores_train, 'grunnkrets_id')




stores_train = replace_missing(stores_train, 'district_name', 'No district')
stores_train = replace_missing(stores_train, 'municipality_name' , "No municipality_name")
stores_train = replace_missing(stores_train, 'area_km2' , 0.0)



#stores_train = replace_missing(stores_train, 'stores_in_mall', 0.0)
#stores_train = replace_missing(stores_train, 'chain_num', 0.0)
#stores_train = replace_missing(stores_train, 'sum_people', 0.0)

stores_test = replace_missing(stores_test, 'district_name', 'No district')
stores_test = replace_missing(stores_test, 'municipality_name' , "No municipality_name")
stores_test = replace_missing(stores_test, 'area_km2' , 0.0)

stores_test = replace_missing(stores_test, "lv3_avg_rev", MEAN_REVENUE)
stores_test = replace_missing(stores_test, "lv4_avg_rev", MEAN_REVENUE)



#stores_train = replace_missing(stores_train, 'sum_people', 0.0)
#eda.create_report(stores_train)



#stores_train = remove_column(stores_train, "plaace_hierarchy_id")
#stores_test  = remove_column(stores_test, "plaace_hierarchy_id")
#stores_train = remove_column(stores_train, "store_id")
#stores_test  = remove_column(stores_test, "store_id")


print(stores_train.head(3).T)
#eda.create_report(stores_train)

                                               0                       1  \
grunnkrets_id                            6020303                 3010306   
chain_name                             MCDONALDS               MCDONALDS   
mall_name                      Magasinet Drammen                 no mall   
lv1_desc                  Dining and Experiences  Dining and Experiences   
lv2_desc                              Restaurant              Restaurant   
lv3_desc                   Hamburger restaurants   Hamburger restaurants   
lv4_desc                   Hamburger restaurants   Hamburger restaurants   
district_name                  Bragernes sentrum               Sentrum 3   
municipality_name                        Drammen                    Oslo   
area_km2                                0.155779                0.264278   
num_of_stores_grunnkrets                   153.0                   230.0   
mall_distance                           0.001483                 0.00413   
closest_mall

In [21]:
stores_train.replace([np.inf, -np.inf], 0, inplace=True)
stores_test.replace([np.inf, -np.inf], 0, inplace=True)

stores_train.columns

Index(['grunnkrets_id', 'chain_name', 'mall_name', 'lv1_desc', 'lv2_desc',
       'lv3_desc', 'lv4_desc', 'district_name', 'municipality_name',
       'area_km2', 'num_of_stores_grunnkrets', 'mall_distance',
       'closest_mall_name', 'avg_rev_big_chain', 'lv3_avg_rev', 'lv4_avg_rev',
       'cluster', 'center_distance', 'revenue'],
      dtype='object')

In [22]:
# separating training data

X_train = stores_train.drop(columns=['revenue'])
y_train = stores_train.revenue
X_test = stores_test

# encode categorical features


categorical_features = ['chain_name',
                        'grunnkrets_id',
                        'municipality_name',
                        "closest_mall_name",
                        'cluster',
                        'lv1_desc',
                        'lv2_desc',
                        'lv3_desc',
                        'lv4_desc',
                        'mall_name',
                        'district_name',
                       ]

for feature in categorical_features:
    X_train[feature] = X_train[feature].astype('category')
    X_test[feature] = X_test[feature].astype('category')

In [23]:
# LabelEncoding for non-catboost models
le = LabelEncoder()
X_train_cat = X_train.copy()
X_test_cat = X_test.copy()
y_train_cat = y_train.copy()


X_train_shape = X_train.shape[0]
X_test_shape = X_test.shape[0]

X_train["train_test_split"] = [0 for i in range(X_train_shape)]
X_test["train_test_split"] = [1 for i in range(X_test_shape)]

X_train_test = X_train.append(X_test, ignore_index=True)

X_train_test[categorical_features] = X_train_test[categorical_features].apply(le.fit_transform)
X_train = X_train_test.loc[X_train_test["train_test_split"] == 0]
X_test = X_train_test.loc[X_train_test["train_test_split"] == 1]

X_train=X_train.drop(["train_test_split"], axis=1)
X_test=X_test.drop(["train_test_split"], axis=1)

In [24]:
class RMSLEMetric(object):
    def get_final_error(self, error, weight):
        return error

    def is_max_optimal(self):
        # the larger metric value the better
        return False

    def evaluate(self, approxes, target, weight):
        assert len(approxes) == 1
        assert len(target) == len(approxes[0])
        preds = np.array(approxes[0])
        target = np.array(target)
        preds = np.exp(preds) - 1
        target = np.exp(target) - 1
        for i in range(len(preds)):
            if preds[i]<0:
                preds[i] = 0
        assert (target >= 0).all(), 'Received negative target values'
        assert (preds >= 0).all(), 'Received negative pred values'
        assert target.shape == preds.shape, 'target and pred have different shapes'
        target_log1p = np.log1p(target)  # log(1 + y_true)
        preds_log1p = np.log1p(preds)  # log(1 + y_pred)
        
        return np.sqrt(np.mean(np.square(preds_log1p - target_log1p))), 0


def rmsle(y_true, y_pred):
    """
    Computes the Root Mean Squared Logarithmic Error 
    
    Args:
        y_true (np.array): n-dimensional vector of ground-truth values 
        y_pred (np.array): n-dimensional vecotr of predicted values 
    
    Returns:
        A scalar float with the rmsle value 
    
    Note: You can alternatively use sklearn and just do: 
        `sklearn.metrics.mean_squared_log_error(y_true, y_pred) ** 0.5`
    """
    for i in range(len(y_pred)):
        if y_pred[i]<0:
            y_pred[i] = 0
    assert (y_true >= 0).all(), 'Received negative y_true values'
    assert (y_pred >= 0).all(), 'Received negative y_pred values'
    assert y_true.shape == y_pred.shape, 'y_true and y_pred have different shapes'
    y_true = np.exp(y_true) - 1
    y_pred = np.exp(y_pred) - 1
    y_true_log1p = np.log1p(y_true)  # log(1 + y_true)
    y_pred_log1p = np.log1p(y_pred)  # log(1 + y_pred)
    print(np.sqrt(np.mean(np.square(y_pred_log1p - y_true_log1p))))
    return np.sqrt(np.mean(np.square(y_pred_log1p - y_true_log1p)))


In [25]:
#eda.create_report(X_test)

In [26]:
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
import xgboost as xgb
import h2o
from h2o.automl import H2OAutoML

cat_params =     {
    "random_state":8,
    "learning_rate":0.01,
    "max_depth":8,
    "eval_metric":RMSLEMetric(),
    "cat_features":categorical_features,
    "n_estimators":3000,
    "od_type":"Iter",
    "silent":True
}

random_forest_params = {
    "n_estimators":100,
    "max_depth":9,
    "random_state":8,
    
    "max_features":None,
}

light_gbm_params = {
    "num_leaves":8,
    "max_depth":8, 
    "random_state":8,
    "silent":True, 
    "metric":'rmsle',
    "n_jobs":-1,
    #"min_data_in_leaf": 20,
    "lambda_l1": 0.5,
    "n_estimators":1500,
    "colsample_bytree":0.95,
    "subsample":0.2,
    "learning_rate":0.008,
}
xgboost_params = {
            'lambda': 2.5 ,#2.5463783373378037,
             'alpha': 0.0023357293126326307,
             'colsample_bytree': 1.0,
             'subsample': 0.6, #0.6,
             'learning_rate': 0.005, #0.012,
             'max_depth': 5, #4,
             'random_state': 8,
             'min_child_weight': 26,
             #"enable_categorical":"True",
            'n_estimators': 1500}

model_catboost= CatBoostRegressor(**cat_params)
model_random_forest = RandomForestRegressor(**random_forest_params)
model_light_gbm = lgb.LGBMRegressor(**light_gbm_params)
model_xgboost = xgb.XGBRegressor(**xgboost_params)

model_catboost.fit(X_train, y_train)
model_random_forest.fit(X_train, y_train)
model_light_gbm.fit(X_train, y_train)
model_xgboost.fit(X_train, y_train)





# model_random_forest.fit(X_train, y_train)
# print("fitted")
# cv = KFold(n_splits = 4, random_state=42, shuffle=True)
# score = cross_val_score(model_random_forest, X_train, y_train, cv=cv, scoring=make_scorer(rmsle))

# print("scores:",score,"\n")
# print("mean:\n",np.mean(score))

pred_1 = np.exp(model_catboost.predict(X_test)) -1
pred_2 = np.exp(model_random_forest.predict(X_test)) -1
pred_3 = np.exp(model_light_gbm.predict(X_test)) -1
pred_4 = np.exp(model_xgboost.predict(X_test)) -1

# print(pred[:12], "\n", pred[-5:])
''' 
xgboost:

n_estimators:
2000 - 0.6798
1000 - 0.6798

lambda:
2.54.. - 0.67977
1 - 0.67980 
3 - 0.67984
2 - 0.
2.5 - 0.67975

subsample:
1 - 0.682
0.5 - 0.67984
0.7 - 0.67988
0.6 - 0.67975


estimators: 
4000 - 6.8...
1500 - 0.6791
'''
preds=[pred_1,pred_2,pred_3,pred_4]
for i in range(4):
    print(pd.DataFrame(preds[i]).describe())

/lhome/frimanng/.local/lib/python3.10/site-packages/catboost/core.py:1759: UserWarning: Failed to optimize method "evaluate" in the passed object:
Failed in nopython mode pipeline (step: nopython frontend)
No implementation of function Function(<built-in function array>) found for signature:
 
 >>> array(array(float64, 1d, C))
 
There are 4 candidate implementations:
   - Of which 4 did not match due to:
   Overload in function '_OverloadWrapper._build.<locals>.ol_generated': File: numba/core/overload_glue.py: Line 131.
     With argument(s): '(array(float64, 1d, C))':
    Rejected as the implementation raised a specific error:
      TypingError: Failed in nopython mode pipeline (step: nopython frontend)
    No implementation of function Function(<intrinsic stub>) found for signature:
     
     >>> stub(array(float64, 1d, C))
     
    There are 2 candidate implementations:
      - Of which 2 did not match due to:
      Intrinsic in function 'stub': File: numba/core/overload_glue.py: 

[LightGBM] [Warning] lambda_l1 is set=0.5, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5
                 0
count  8577.000000
mean      6.464397
std      10.490753
min       0.467144
25%       2.023454
50%       3.218689
75%       5.322310
max      93.860871
                 0
count  8577.000000
mean      6.444790
std      10.647843
min       0.600460
25%       2.083118
50%       3.323108
75%       5.238721
max      79.038356
                 0
count  8577.000000
mean      6.326476
std      10.033677
min       0.359554
25%       2.056633
50%       3.260302
75%       5.284761
max      76.558748
                 0
count  8577.000000
mean      6.390781
std      10.375998
min       0.528633
25%       2.041240
50%       3.245517
75%       5.311893
max      75.513023


In [27]:
''' 


print("#")
'''
features_for_model = [f for f in X_train]
feat_import = [t for t in zip(features_for_model,model_random_forest.feature_importances_)]
feat_import_df = pd.DataFrame(feat_import, columns = ['Features', 'Random_Forest_importance'])
df_rf =feat_import_df.sort_values("Random_Forest_importance", ascending= False)

features_for_model = [f for f in X_train]
feat_import = [t for t in zip(features_for_model,model_catboost.get_feature_importance())]
feat_import_df = pd.DataFrame(feat_import, columns = ['Features', 'CatBoost_importance'])
df_cat =feat_import_df.sort_values("CatBoost_importance", ascending= False)
df_cat



features_for_model = [f for f in X_train]
feat_import = [t for t in zip(features_for_model,model_light_gbm.feature_importances_)]
feat_import_df = pd.DataFrame(feat_import, columns = ['Features', 'Ligt_gbm_importance'])
df_lgb =feat_import_df.sort_values("Ligt_gbm_importance", ascending= False)
df_lgb = pd.merge(df_lgb,df_cat, how="left", on="Features")

df_lgb

features_for_model = [f for f in X_train]
feat_import = [t for t in zip(features_for_model,model_xgboost.feature_importances_)]
feat_import_df = pd.DataFrame(feat_import, columns = ['Features', 'xgboost_importance'])
df_xgb =feat_import_df.sort_values("xgboost_importance", ascending= False)
df = pd.merge(df_xgb,df_lgb,how="left", on="Features")
df = pd.merge(df, df_rf, how="left", on="Features")
df


,Features,xgboost_importance,Ligt_gbm_importance,CatBoost_importance,Random_Forest_importance
0,avg_rev_big_chain,0.514971,1535,26.809333,0.688675
1,lv4_avg_rev,0.196445,608,6.290917,0.117929
2,lv3_avg_rev,0.039564,670,2.738361,0.014992
3,mall_name,0.025018,388,3.353332,0.006927
4,center_distance,0.024789,864,2.740349,0.025299
5,num_of_stores_grunnkrets,0.023212,918,5.632760,0.023312
6,lv1_desc,0.022900,142,3.223451,0.000888
7,mall_distance,0.020345,1082,5.243051,0.034782
8,lv3_desc,0.015965,503,4.588549,0.003524
9,lv2_desc,0.015618,336,4.594739,0.004452


In [28]:
cv = KFold(n_splits = 4, random_state=42, shuffle=True)
score_lgbm = cross_val_score(model_light_gbm, X_train, y_train, cv=cv, scoring=make_scorer(rmsle))
score_random_forest = cross_val_score(model_random_forest, X_train, y_train, cv=cv, scoring=make_scorer(rmsle))
score_cat = cross_val_score(model_catboost, X_train, y_train, cv=cv, scoring=make_scorer(rmsle))
score_xgboost = cross_val_score(model_xgboost, X_train, y_train, cv=cv, scoring=make_scorer(rmsle))

print('Scores for all folds: ')
print("lgbm:", score_lgbm)
print("cat:", score_cat)
print("xg boost:", score_xgboost)
print("random forest:", score_random_forest)
print('\n')
print('Mean of scores:')
print("lgbm:", np.mean(score_lgbm))
print("cat:", np.mean(score_cat))
print("xg boost:", np.mean(score_xgboost))
print("random forest:", np.mean(score_random_forest))
print('\n')
print('Standard deviation of scores: ')
print(np.std(score_lgbm))
print(np.std(score_cat))
print(np.std(score_xgboost))

/lhome/frimanng/.local/lib/python3.10/site-packages/lightgbm/sklearn.py:598: UserWarning: 'silent' argument is deprecated and will be removed in a future release of LightGBM. Pass 'verbose' parameter via keyword arguments instead.
  _log_warning("'silent' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] lambda_l1 is set=0.5, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5
0.6695972314836428
[LightGBM] [Warning] lambda_l1 is set=0.5, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5
0.665783704608548
[LightGBM] [Warning] lambda_l1 is set=0.5, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5
0.6782971922195811
[LightGBM] [Warning] lambda_l1 is set=0.5, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5
0.6729265176710479
0.6741360410481396
0.6731674382501849
0.6831965392934801
0.6724105760218367


/lhome/frimanng/.local/lib/python3.10/site-packages/catboost/core.py:1759: UserWarning: Failed to optimize method "evaluate" in the passed object:
Failed in nopython mode pipeline (step: nopython frontend)
No implementation of function Function(<built-in function array>) found for signature:
 
 >>> array(array(float64, 1d, C))
 
There are 4 candidate implementations:
      - Of which 4 did not match due to:
      Overload in function '_OverloadWrapper._build.<locals>.ol_generated': File: numba/core/overload_glue.py: Line 131.
        With argument(s): '(array(float64, 1d, C))':
       Rejected as the implementation raised a specific error:
         TypingError: Failed in nopython mode pipeline (step: nopython frontend)
       No implementation of function Function(<intrinsic stub>) found for signature:
        
        >>> stub(array(float64, 1d, C))
        
       There are 2 candidate implementations:
         - Of which 2 did not match due to:
         Intrinsic in function 'stub':

0.6668964569929188


/lhome/frimanng/.local/lib/python3.10/site-packages/catboost/core.py:1759: UserWarning: Failed to optimize method "evaluate" in the passed object:
Failed in nopython mode pipeline (step: nopython frontend)
No implementation of function Function(<built-in function array>) found for signature:
 
 >>> array(array(float64, 1d, C))
 
There are 4 candidate implementations:
      - Of which 4 did not match due to:
      Overload in function '_OverloadWrapper._build.<locals>.ol_generated': File: numba/core/overload_glue.py: Line 131.
        With argument(s): '(array(float64, 1d, C))':
       Rejected as the implementation raised a specific error:
         TypingError: Failed in nopython mode pipeline (step: nopython frontend)
       No implementation of function Function(<intrinsic stub>) found for signature:
        
        >>> stub(array(float64, 1d, C))
        
       There are 2 candidate implementations:
         - Of which 2 did not match due to:
         Intrinsic in function 'stub':

0.6638102163036352


/lhome/frimanng/.local/lib/python3.10/site-packages/catboost/core.py:1759: UserWarning: Failed to optimize method "evaluate" in the passed object:
Failed in nopython mode pipeline (step: nopython frontend)
No implementation of function Function(<built-in function array>) found for signature:
 
 >>> array(array(float64, 1d, C))
 
There are 4 candidate implementations:
      - Of which 4 did not match due to:
      Overload in function '_OverloadWrapper._build.<locals>.ol_generated': File: numba/core/overload_glue.py: Line 131.
        With argument(s): '(array(float64, 1d, C))':
       Rejected as the implementation raised a specific error:
         TypingError: Failed in nopython mode pipeline (step: nopython frontend)
       No implementation of function Function(<intrinsic stub>) found for signature:
        
        >>> stub(array(float64, 1d, C))
        
       There are 2 candidate implementations:
         - Of which 2 did not match due to:
         Intrinsic in function 'stub':

0.6741096068033473


/lhome/frimanng/.local/lib/python3.10/site-packages/catboost/core.py:1759: UserWarning: Failed to optimize method "evaluate" in the passed object:
Failed in nopython mode pipeline (step: nopython frontend)
No implementation of function Function(<built-in function array>) found for signature:
 
 >>> array(array(float64, 1d, C))
 
There are 4 candidate implementations:
      - Of which 4 did not match due to:
      Overload in function '_OverloadWrapper._build.<locals>.ol_generated': File: numba/core/overload_glue.py: Line 131.
        With argument(s): '(array(float64, 1d, C))':
       Rejected as the implementation raised a specific error:
         TypingError: Failed in nopython mode pipeline (step: nopython frontend)
       No implementation of function Function(<intrinsic stub>) found for signature:
        
        >>> stub(array(float64, 1d, C))
        
       There are 2 candidate implementations:
         - Of which 2 did not match due to:
         Intrinsic in function 'stub':

0.661792312953942
0.6675319038673946
0.6653077661447461
0.6750166554228
0.669309114205358
Scores for all folds: 
lgbm: [0.66959723 0.6657837  0.67829719 0.67292652]
cat: [0.66689646 0.66381022 0.67410961 0.66179231]
xg boost: [0.6675319  0.66530777 0.67501666 0.66930911]
random forest: [0.67413604 0.67316744 0.68319654 0.67241058]


Mean of scores:
lgbm: 0.6716511614957048
cat: 0.6666521482634609
xg boost: 0.6692913599100747
random forest: 0.6757276486534103


Standard deviation of scores: 
0.004594614689334306
0.004673541246456987
0.003596666122130395


In [29]:
print('Mean of scores:')
print("lgbm:", np.mean(score_lgbm))
print("cat:", np.mean(score_cat))
print("xg boost:", np.mean(score_xgboost))
print("random forest:", np.mean(score_random_forest))

Mean of scores:
lgbm: 0.6716511614957048
cat: 0.6666521482634609
xg boost: 0.6692913599100747
random forest: 0.6757276486534103


In [30]:
# mean of scores
# 0.6781866272821699 - 11-12-123108 - kaggle: 0.68349

# mean of scores 3- stack
# 6781866272821699 / 0.6839925788085071 / 0.679119033159766 - 11-12-132139 - kaggle: 0.68413

# mean of scores models:
# 0.6976428658736472 - random forest - kaggle: 0.70203
# 0.6857052809131627 - random forest - new hyper params

# mean of scores after adding rival_revenue
# lgbm: 0.6824655631333725 
# cat:  0.6769482640447066 - with mean => kaggle: 0.68335

# mean of scores after removing rival_revenue, adding rival_distance
# lgbm: 0.6819900731158717
# cat: 0.6778082165931306 - with mean => kaggle: 0.68290


# Mean of scores: - changed avg avg to not >3
# lgbm: 0.6726921524863859
# cat: 0.6695708730130597
# xg boost: 0.6711098372410809


# Mean of scores: - changed outliers of removing rev>150
# lgbm: 0.6716511614957048
# cat: 0.6666521482634609
# xg boost: 0.6692913599100747
# random forest: 0.6757276486534103

# switched avg_rev_chain back to >3, removing overall rev > 100 (removed around 70 stores, where 40 are rema 1000)
# Mean of scores:
# lgbm: 0.6752373567799875
# cat: 0.6725937329378754
# xg boost: 0.672993772025054
# random forest: 0.6794574168532559

# Mean of scores: #removed rev>135, added bus stops
# lgbm: 0.6785290668037811
# cat: 0.6741125811018365
# xg boost: 0.6753203790091535
# random forest: 0.6820343581205036

# Mean of scores:  => busstop distance =500meters
# lgbm: 0.6785512739369779
# cat: 0.6740236184052476
# xg boost: 0.675544169615468
# random forest: 0.6819594860144613


X_test = replace_missing(X_test, "lv3_avg_rev", MEAN_REVENUE)
X_test = replace_missing(X_test, "lv4_avg_rev", MEAN_REVENUE)


catboost_prediction = model_catboost.predict(X_test)
light_gbm_prediction= model_light_gbm.predict(X_test)
random_forest_prediction = model_random_forest.predict(X_test)
xgboost_prediction = model_xgboost.predict(X_test)

#all_predictions= np.array([random_forest_prediction])#np.array([catboost_prediction, light_gbm_prediction, random_forest_prediction])
all_predictions = np.array([catboost_prediction, light_gbm_prediction, xgboost_prediction, random_forest_prediction])
for prd in all_predictions:
    print( np.exp(prd[:12]) -1 )
    print(np.mean(np.exp(prd)-1))
    #print(pd.DataFrame(np.exp(prd) -1 ).describe())



final_prediction = np.mean( all_predictions , axis=0)
for i in range(len(final_prediction)):
    if final_prediction[i]<0:
        final_prediction[i] = 0
        
final_prediction = np.exp(final_prediction) - 1

[ 3.03669231  5.76493326  5.25375659  6.77026178 31.06777182  6.03123552
 20.35538613  6.55554674 21.67773028 19.20102285  4.79972115  4.73765937]
6.464396694313658
[ 3.98361459  6.58738719  5.63518013  6.96839946 29.36479051  5.8380423
 25.89240166  6.78107016 25.40994032 25.42546808  5.65797146  5.49791283]
6.326476155790505
[ 3.93491893  6.50860087  4.54215024  7.01963164 30.61977756  5.64603818
 28.21770013  7.17047891 26.29401174 25.03039636  5.67054537  4.77274716]
6.390769885543436
[ 3.95405003  6.21309296  4.51043676  6.47797904 29.9116488   4.89288311
 26.7265181   6.12325547 23.85612631 27.37715527  4.33000068  5.07873032]
6.444790037063013


In [31]:
# generate .csv-submission


submission = pd.DataFrame()
submission['id'] = stores_test_id
submission['predicted'] = np.asarray(final_prediction)

from datetime import datetime
now = datetime.now() # current date and time
timestamp = now.strftime("%Y-%m-%d_%H%M%S")

path = '../predictions'
prefix = 'submission'
suffix = '.csv'

filename = "".join(["-".join([prefix,timestamp]),suffix])

#eda.create_report(submission)
#submission.to_csv("/".join([path,filename]), index=False)

print(submission[:12], submission[-5:])
submission.describe()


# same as submission-2022-11-13-155127

                            id  predicted
0   914206820-914239427-717245   3.709260
1   916789157-916823770-824309   6.261211
2     913341082-977479363-2948   4.966429
3    889682582-889697172-28720   6.806140
4   997991699-998006945-417222  30.234173
5   914931487-815162862-756427   5.587392
6     967062979-972338656-6209  25.116778
7   914631734-914748119-740036   6.648175
8    970976361-973961837-23171  24.247417
9    979425031-979584385-54031  24.058866
10  914852625-914864976-744489   5.087086
11  916756097-816761972-821991   5.014174                               id  predicted
8572  917323003-917383529-844309   7.364807
8573  917353379-917411824-845904   3.414573
8574  917072302-917089248-833647   7.652538
8575  916960557-916993161-829908   2.555419
8576   987280891-972040746-45320   2.717629


,predicted
count,8577.000000
mean,6.383359
std,10.298164
min,0.579663
25%,2.064235
50%,3.262634
75%,5.242266
max,77.512161
